# 02 - Funciones en Python

**Autor:** Miguel Ángel Vázquez Varela  
**Nivel:** Fundamentos  
**Tiempo estimado:** 25 min

---

## ¿Qué aprenderás?

- Crear funciones reutilizables
- Parámetros y valores de retorno
- Argumentos por defecto y keyword arguments
- Scope de variables (local vs global)
- Documentación con docstrings

---

## 1. ¿Por qué funciones?

Las funciones permiten:
- **Reutilizar** código (DRY: Don't Repeat Yourself)
- **Organizar** lógica en bloques con nombre
- **Testear** partes del código independientemente
- **Abstraer** complejidad

In [ ]:
# ❌ Sin funciones: código repetido
trip_duration_1 = 15
trip_duration_2 = 22
trip_duration_3 = 8

# Calcular si es viaje largo (>20 min) para cada uno
is_long_1 = trip_duration_1 > 20
is_long_2 = trip_duration_2 > 20
is_long_3 = trip_duration_3 > 20

print(f"Viaje 1 es largo: {is_long_1}")
print(f"Viaje 2 es largo: {is_long_2}")
print(f"Viaje 3 es largo: {is_long_3}")

In [ ]:
# ✅ Con función: reutilizable
def is_long_trip(duration_minutes, threshold=20):
    """Determina si un viaje es largo basándose en su duración."""
    return duration_minutes > threshold

# Ahora es fácil de usar y modificar
print(f"Viaje 1 es largo: {is_long_trip(15)}")
print(f"Viaje 2 es largo: {is_long_trip(22)}")
print(f"Viaje 3 es largo: {is_long_trip(8)}")

# Puedo cambiar el threshold fácilmente
print(f"\nCon threshold de 10 min:")
print(f"Viaje 1 es largo: {is_long_trip(15, threshold=10)}")

---

## 2. Anatomía de una función

```python
def nombre_funcion(param1, param2=valor_defecto):
    """Docstring: describe qué hace la función."""
    # Cuerpo de la función
    resultado = param1 + param2
    return resultado
```

In [ ]:
def calculate_trip_speed(distance_km, duration_minutes):
    """
    Calcula la velocidad media de un viaje.
    
    Parameters
    ----------
    distance_km : float
        Distancia del viaje en kilómetros
    duration_minutes : float
        Duración del viaje en minutos
    
    Returns
    -------
    float
        Velocidad en km/h
    
    Examples
    --------
    >>> calculate_trip_speed(5, 30)
    10.0
    """
    if duration_minutes <= 0:
        return 0
    
    duration_hours = duration_minutes / 60
    speed_kmh = distance_km / duration_hours
    
    return speed_kmh

# Usar la función
speed = calculate_trip_speed(distance_km=5, duration_minutes=30)
print(f"Velocidad: {speed} km/h")

# Ver la documentación
help(calculate_trip_speed)

---

## 3. Parámetros: posicionales vs keyword

In [ ]:
def describe_station(name, city, bikes, docks, active=True):
    """Genera una descripción de una estación de bicicletas."""
    status = "activa" if active else "inactiva"
    return f"{name} ({city}): {bikes}/{docks} bicis - {status}"

# Argumentos posicionales (por orden)
print(describe_station("Sol", "Madrid", 15, 30))

# Keyword arguments (por nombre) - más legible
print(describe_station(
    name="Atocha",
    city="Madrid",
    bikes=8,
    docks=25,
    active=False
))

# Mezclando (posicionales primero, luego keywords)
print(describe_station("Cibeles", "Madrid", bikes=20, docks=40))

### ⚠️ Cuidado con argumentos mutables por defecto

In [ ]:
# ❌ PELIGRO: lista mutable como valor por defecto
def add_trip_bad(trip_id, trips=[]):
    trips.append(trip_id)
    return trips

print(add_trip_bad(1))  # [1]
print(add_trip_bad(2))  # [1, 2] - ¡La lista persiste!
print(add_trip_bad(3))  # [1, 2, 3] - ¡Bug!

print("---")

# ✅ CORRECTO: usar None y crear dentro
def add_trip_good(trip_id, trips=None):
    if trips is None:
        trips = []
    trips.append(trip_id)
    return trips

print(add_trip_good(1))  # [1]
print(add_trip_good(2))  # [2] - Correcto

---

## 4. Retornando múltiples valores

In [ ]:
def analyze_trips(durations):
    """
    Analiza una lista de duraciones de viaje.
    
    Returns
    -------
    tuple
        (count, total, average, min_val, max_val)
    """
    if not durations:
        return 0, 0, 0, None, None
    
    count = len(durations)
    total = sum(durations)
    average = total / count
    min_val = min(durations)
    max_val = max(durations)
    
    return count, total, average, min_val, max_val

# Datos de ejemplo
trip_durations = [12, 25, 8, 45, 15, 30, 18]

# Desempaquetar todos los valores
count, total, avg, min_d, max_d = analyze_trips(trip_durations)

print(f"Número de viajes: {count}")
print(f"Duración total: {total} min")
print(f"Duración media: {avg:.1f} min")
print(f"Rango: {min_d} - {max_d} min")

In [ ]:
# Alternativa: retornar diccionario (más explícito)
def analyze_trips_dict(durations):
    """Retorna estadísticas como diccionario."""
    if not durations:
        return {"count": 0, "total": 0, "average": 0}
    
    return {
        "count": len(durations),
        "total": sum(durations),
        "average": sum(durations) / len(durations),
        "min": min(durations),
        "max": max(durations)
    }

stats = analyze_trips_dict(trip_durations)
print(f"Media: {stats['average']:.1f} min")
print(f"Rango: {stats['min']} - {stats['max']} min")

---

## 5. Scope: variables locales vs globales

In [ ]:
# Variable global
DEFAULT_THRESHOLD = 20

def is_long_trip_v2(duration):
    # Puede LEER variables globales
    return duration > DEFAULT_THRESHOLD

print(is_long_trip_v2(25))  # True

def bad_function():
    # Crea una variable LOCAL, no modifica la global
    DEFAULT_THRESHOLD = 100
    print(f"Dentro de la función: {DEFAULT_THRESHOLD}")

bad_function()
print(f"Fuera de la función: {DEFAULT_THRESHOLD}")  # Sigue siendo 20

In [ ]:
# ✅ Mejor práctica: pasar valores como parámetros
def is_long_trip_clean(duration, threshold):
    """No depende de variables globales - más predecible y testeable."""
    return duration > threshold

# El comportamiento es explícito
print(is_long_trip_clean(25, threshold=20))  # True
print(is_long_trip_clean(25, threshold=30))  # False

---

## 6. Funciones Lambda (anónimas)

Funciones pequeñas de una línea, útiles para operaciones simples.

In [ ]:
# Función normal
def double(x):
    return x * 2

# Equivalente lambda
double_lambda = lambda x: x * 2

print(f"Normal: {double(5)}")
print(f"Lambda: {double_lambda(5)}")

# Útil con sorted(), map(), filter()
stations = [
    {"name": "Sol", "bikes": 15},
    {"name": "Atocha", "bikes": 8},
    {"name": "Cibeles", "bikes": 22}
]

# Ordenar por número de bicis
sorted_stations = sorted(stations, key=lambda s: s["bikes"])
print(f"\nOrdenadas por bicis:")
for s in sorted_stations:
    print(f"  {s['name']}: {s['bikes']}")

---

## 💡 Resumen

| Concepto | Ejemplo |
|----------|--------|
| Definición | `def func(param):` |
| Parámetro por defecto | `def func(x=10):` |
| Keyword arguments | `func(x=5, y=10)` |
| Múltiples retornos | `return a, b, c` |
| Docstring | `"""Descripción"""` |
| Lambda | `lambda x: x * 2` |

**Buenas prácticas:**
- Nombres descriptivos (`calculate_speed`, no `calc`)
- Una función = una responsabilidad
- Documenta con docstrings
- Evita mutables como valores por defecto
- Prefiere parámetros explícitos sobre variables globales

---

## 🏋️ Ejercicio

Crea una función `classify_trip(duration_minutes)` que:
- Retorne `"corto"` si < 10 min
- Retorne `"medio"` si 10-30 min
- Retorne `"largo"` si > 30 min

Bonus: permite personalizar los umbrales con parámetros opcionales.

In [ ]:
# Tu código aquí
def classify_trip(duration_minutes, short_threshold=10, long_threshold=30):
    """Clasifica un viaje por su duración."""
    # Implementa la lógica
    pass

# Tests
print(classify_trip(5))   # "corto"
print(classify_trip(15))  # "medio"
print(classify_trip(45))  # "largo"

---

**Anterior:** [01 - Variables y Tipos](../01_python_fundamentals/01_variables_and_types.ipynb)  
**Siguiente:** [03 - NumPy Essentials](../03_numpy_essentials/01_arrays_basics.ipynb)